In [ ]:
# Install xlsxwriter & openpyxl if you don't have it
# !pip install xlsxwriter openpyxl

import pandas as pd

PATH_DATA = '../'
# Create a sample dataframe
data = pd.read_excel(PATH_DATA+'Folios_boton_promad_2023-02-12.xlsx', dtype=str, sheet_name='Folios')
data.head()

## Tabla - Códigos de Cierre

### Creación

In [ ]:
# Create a pivot table
pivot_table = pd.pivot_table(
    data[data['C2_INICIO']=='C2 PONIENTE'], 
    values='FOLIO', 
    index='CODIGO_MOTIVO_CIERRE_INSTITUCION', 
    columns=['mes_cierre', 'sin_corregir'], 
    margins=True, margins_name='TOTAL',
    aggfunc='count', fill_value=0
)

# Drop any coloumns where pair is (Any, 'Corregido')
pivot_table = pivot_table.drop(columns='Corregido', level=1)
# Sum again column TOTAL to get the total of the row
pivot_table['TOTAL'] = pivot_table.sum(axis=1)
# Rename index to Código de cierre
pivot_table.index.name = 'Código de Cierre'

In [ ]:
# Generating the table content

pivot_table_final = pivot_table.loc[:'SYS'][:-1]
pivot_table_final.loc['SUB TOTAL'] = pivot_table_final.sum()
pivot_table_final = pd.concat([pivot_table_final, pivot_table.loc['SYS':]])
pivot_table = pivot_table_final

# Sum as int
pivot_table['TOTAL'] = pivot_table.drop(columns='TOTAL').sum(axis=1).astype(int)
pivot_table.reset_index(inplace=True)
print(pivot_table.columns)
pivot_table

### Formato

In [ ]:
# Formatting the table

# 1. Set columns in level 2 Months from level 1 in uppercase only first 3 letters
pivot_table.columns = pd.MultiIndex.from_tuples(
    [((col[1], col[0][:3].upper()) if (i+1<len(pivot_table.columns) and i>0) else (col[0],col[1])) for i, col in enumerate(pivot_table.columns)]
)

# 2. Rename columns in level 0 where 'Sin Corregir' to 'Sin Reclasificar'
pivot_table.columns = pd.MultiIndex.from_tuples(
    [('Sin Reclasificar', col[1]) if col[0]=='Sin corregir' else col for col in pivot_table.columns]
)
pivot_table

In [ ]:
pivot_table.index

In [ ]:

# # Write the pivot table to an Excel file
writer = pd.ExcelWriter('pivot_table.xlsx', engine='xlsxwriter')
pivot_table.to_excel(writer, sheet_name='Tabla Código de Cierre')

workbook = writer.book
worksheet = writer.sheets['Tabla Código de Cierre']

# Drop column A 
worksheet.set_column(0, 0, 1)
# Merge cells from A1 to A2 if A2 is not empty

# for i in range(1, len(pivot_table.index)):
# 	if pivot_table.iloc[i,1] != '':
# 		worksheet.merge_range(f'A{i+1}:A{i+2}', pivot_table.iloc[i,0], workbook.add_format({'bold': True, 'align': 'center', 'valign': 'vcenter'}))



writer.save()

In [ ]:


import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side

PATH_DATA = '../'
file = 'Folios_boton_promad_2023-02-12.xlsx'

# Leer el archivo excel
file = "Folios_boton_promad_2023-02-12.xlsx"
df = pd.read_excel(PATH_DATA+file, sheet_name="Folios")#, dtype=str)

# Crear la tabla dinámica
pivot_table = df.pivot_table(
    index="C2_INICIO",
    columns=["mes_cierre", "sin_corregir"],
    values="FOLIO",
    aggfunc="count",
    fill_value=0,
    margins=True,
    margins_name="Total general",
)

# Guardar la tabla dinámica en un nuevo archivo Excel
output_file = "PivotTable_CodigosCierre.xlsx"
pivot_table.to_excel(output_file)

# Aplicar la personalización requerida
wb = load_workbook(output_file)
ws = wb.active

# Personalizar celdas 'TOTAL' y 'SIN RECLASIFICAR'
total_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")
sin_reclasificar_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

ws.merge_cells("B1:E1")
ws["B1"] = "TOTAL"
ws["B1"].fill = total_fill

ws.merge_cells("F1:H1")
ws["F1"] = "SIN RECLASIFICAR"
ws["F1"].fill = sin_reclasificar_fill

# Resaltar filas según la variable RESALTAR
resaltar = "C2 NORTE"  # Cambia esta variable según tus necesidades
border = Border(left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin"))

for row in ws.iter_rows(min_row=2, max_row=7):
    if row[0].value == resaltar:
        for cell in row:
            cell.border = border

# Guardar los cambios
wb.save(output_file)
